# Optimal Office Hours

## Description

Find the optimal walk-in office hours given every student's availability.

- Poll every student's availability using the free online scheduling tool [doodle.com](URL). Export the result as an excel spreadsheet.
- Use `doodle.ipynb` to rank every possible combination of office hours based on how many students can attend them.

**Ranking algorithm**: Office hours schedules are first ranked based on the number of students who can attend at least one office hour. In case of tie, the number of students who can attend at least two slots is used. In case of tie, the number of students who can attend at least three slots is used. And so on.

**Caveat 1**: The ranking algorithm only works well if students share every time slot they can in principle attend. If a student only shares their favorite slot, the schedule ends up being built around it.

**Caveat 2**: The computation time grows very quickly with the number of time slots. For example, the algorithm check about 200,000 options to find the best 6 out of 25 potential slots, but about 16,000,000 to find the best 6 out of 50.

## Author

Yaouen Fily

In [1]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import itertools as itt
from datetime import datetime as dt

In [2]:
data = pd.read_excel('Doodle.xls',header=[4,5],index_col=0).iloc[:-1]
# display(data[:5])
# print(data.shape)

# Convert a (day,time) tuple to a (compact) string.
def format_slot(slot):
    day,time = slot
    return day[:3]+' '+time.replace(':00 ','').replace(' – ','-').lower()

data.columns       = map(format_slot,data.columns)
data[data=='OK']   = 1
data[data=='(OK)'] = 1
data[data.isna()]  = 0
data               = data.astype(int)

display(data[:5])

,Mon 11am-11:30 am,Mon 11:30 am-12pm,Mon 1pm-1:30 pm,Mon 1:30 pm-2pm,Mon 2pm-2:30 pm,Mon 2:30 pm-3pm,Mon 3pm-3:30 pm,Mon 3:30 pm-4pm,Mon 4pm-4:30 pm,Mon 4:30 pm-5pm,Mon 5pm-5:30 pm,Mon 5:30 pm-6pm,Mon 9pm-9:30 pm,Mon 9:30 pm-10pm,Mon 10pm-10:30 pm,Mon 10:30 pm-11pm
student 1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1
0,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
''' Count the number of students available each slot (same as doodle's "count" row). '''

data.sum(axis=0)

Mon 11am-11:30 am    1
Mon 11:30 am-12pm    1
Mon 1pm-1:30 pm      1
Mon 1:30 pm-2pm      2
Mon 2pm-2:30 pm      1
Mon 2:30 pm-3pm      1
Mon 3pm-3:30 pm      1
Mon 3:30 pm-4pm      1
Mon 4pm-4:30 pm      4
Mon 4:30 pm-5pm      4
Mon 5pm-5:30 pm      4
Mon 5:30 pm-6pm      4
Mon 9pm-9:30 pm      3
Mon 9:30 pm-10pm     3
Mon 10pm-10:30 pm    3
Mon 10:30 pm-11pm    3
dtype: int64

In [4]:
def best_schedules(df,N):
    '''
    Make a list of candidates for the best combination of N office hour slots.
    For each combo, output a list of the number of students who can make it to 0,1,...,N 
    of those slots, followed by the list of slots.
    '''

    schedules = []
    
    for C in itt.combinations(range(df.shape[1]),N): # all unique combinations of three columns
        C = list(C)
        # Number of slots each student can attend.
        ns = np.sum(df.values[:,C],axis=1)
        # List of students who can't make any of the slots in C.
        no = df.index[ns==0]
        # Save the number of students who can't make it, the slots, and the list of students who can't make it.    
        schedules.append((df.columns[C].to_list(),no,ns))
    
    schedules = sorted(schedules,key=lambda x: len(x[1]))
    best_schedules = []
    for C,no,ns in schedules:
        if len(no)==len(schedules[0][1]):
            # Number of students who can attend 0, 1, 2, etc of the slots in that combo.
            ct = [np.count_nonzero(ns==i) for i in range(N+1)]
            best_schedules.append((ct,C))
    best_schedules = sorted(best_schedules)

    df = pd.DataFrame([ [ct]+[c for c in C] for ct,C in best_schedules ])    
    return df

In [5]:
''' Define number of office hours slots. Compute the number of 
combinations the algorithm needs to check. '''

N = 4

n = len(list(itt.combinations(range(data.shape[1]),N)))
print(f'There are {n} possible combinations of {N} slots.')

There are 1820 possible combinations of 4 slots.


In [6]:
''' Find best combinations of various numbers of slots. In each case, save the top 1000 as a text file. '''

t0 = dt.now()
df = best_schedules(data,N)
df = df[:1000] # Don't output more than 1000 options.
with open(f'best-schedules.txt','w') as f:
    f.write(df.to_string(index=False,header=False))
print(f'It took {dt.now()-t0} to check every combination.')

It took 0:00:00.298402 to check every combination.
